In [1]:
import os
import random
import numpy as np
import jax
import flax.linen as nn
from jax import grad, value_and_grad
import jax.numpy as jnp
import optax
import datasets
from datasets import load_dataset
import matplotlib.pyplot as plt

In [40]:
dataset = load_dataset("scene_parse_150")
train_dataset, test_dataset, val_dataset  = dataset["train"], dataset["test"], dataset["validation"]
# Number of segmentation classes
num_classes = 150
# Image height and width
height, width = 256, 256

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
class _unet(nn.Module):

    @nn.compact
    def __call__(self, x):
        ########################### Encoders ############################
        # Encoder block 1
        encoder_block1 = nn.Sequential([
            nn.Conv(features=64, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
            nn.Conv(features=64, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
        ])
        encoder1 = encoder_block1(x)
        x = encoder1
        x = nn.max_pool(inputs=x, window_shape=(2, 2), strides=(2, 2), padding="SAME")
        # Encoder block 2
        encoder_block2 = nn.Sequential([
            nn.Conv(features=128, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
            nn.Conv(features=128, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
        ])
        encoder2 = encoder_block2(x)
        x = encoder2
        x = nn.max_pool(inputs=x, window_shape=(2, 2), strides=(2, 2), padding="SAME")
        # Encoder block 3
        encoder_block3 = nn.Sequential([
            nn.Conv(features=256, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
            nn.Conv(features=256, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
        ])
        encoder3 = encoder_block3(x)
        x = encoder3
        x = nn.max_pool(inputs=x, window_shape=(2, 2), strides=(2, 2), padding="SAME")
        # Encoder block 4
        encoder_block4 = nn.Sequential([
            nn.Conv(features=512, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
            nn.Conv(features=512, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
        ])
        encoder4 = encoder_block4(x)
        x = encoder4
        x = nn.max_pool(inputs=x, window_shape=(2, 2), strides=(2, 2), padding="SAME")
        # Encoder block 5
        encoder_block5 = nn.Sequential([
            nn.Conv(features=1024, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
            nn.Conv(features=1024, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,
        ])
        x = encoder_block5(x)


        ########################### Decoders ############################
        # Decoder block 4
        x = nn.ConvTranspose(features=512, kernel_size=(2, 2), strides=(2, 2), padding="SAME")(x)
        x = jax.nn.relu(x)
        x = jnp.concatenate((x, encoder4), axis=-1)
        decoder_block3 = nn.Sequential([
            nn.Conv(features=512, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,   
            nn.Conv(features=512, kernel_size=(3, 3), padding="SAME"), jax.nn.relu, 
        ])  
        x = decoder_block3(x)    
        # Decoder block 3
        x = nn.ConvTranspose(features=256, kernel_size=(2, 2), strides=(2, 2), padding="SAME")(x)
        x = jax.nn.relu(x)
        x = jnp.concatenate((x, encoder3), axis=-1)
        decoder_block3 = nn.Sequential([
            nn.Conv(features=256, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,   
            nn.Conv(features=256, kernel_size=(3, 3), padding="SAME"), jax.nn.relu, 
        ])  
        x = decoder_block3(x)  
        # Decoder block 2
        x = nn.ConvTranspose(features=128, kernel_size=(2, 2), strides=(2, 2), padding="SAME")(x)
        x = jax.nn.relu(x)
        x = jnp.concatenate((x, encoder2), axis=-1)
        decoder_block2 = nn.Sequential([
            nn.Conv(features=128, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,   
            nn.Conv(features=128, kernel_size=(3, 3), padding="SAME"), jax.nn.relu, 
        ])  
        x = decoder_block2(x)  
        # Decoder block 1
        x = nn.ConvTranspose(features=64, kernel_size=(2, 2), strides=(2, 2), padding="SAME")(x)
        x = jax.nn.relu(x)
        x = jnp.concatenate((x, encoder1), axis=-1)
        decoder_block1 = nn.Sequential([
            nn.Conv(features=64, kernel_size=(3, 3), padding="SAME"), jax.nn.relu,   
            nn.Conv(features=64, kernel_size=(3, 3), padding="SAME"), jax.nn.relu, 
            nn.Conv(features=num_classes, kernel_size=(1, 1), padding="SAME"), jax.nn.sigmoid,
        ])  
        x = decoder_block1(x)  

        return x

# Create the model object
unet = _unet()
# Display the model details
dummy_x = np.array(dataset[0]["image"])
# Downsample the image
dummy_x = jax.image.resize(dummy_x, shape=(height, width, 3), method="bilinear")
rng_key = jax.random.PRNGKey(64)
print(unet.tabulate(rng_key, dummy_x))


                                 _unet Summary                                  
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ path          ┃ module        ┃ inputs        ┃ outputs       ┃ params       ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│               │ _unet         │ float32[256,… │ float32[256,… │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ Sequential_0  │ Sequential    │ float32[256,… │ float32[256,… │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ Conv_0        │ Conv          │ float32[256,… │ float32[256,… │ bias:        │
│               │               │               │               │ float32[64]  │
│               │               │               │               │ kernel:      │
│               │               │               │               │ float32[3,3… │
│               │          

In [28]:
def binarize_target(target_ids, num_classes):
    """
    Converts a single array of target IDs to binary form.
    Each ID will be represented in a seperate channel.
    Args:
        target_ids: A single array of class IDs, 
            with values between 0 and num_classes.
        num_classes: Total number of distinct classes.
    Returns:
        A binarized array of class IDs. 
    """
    (dim_x, dim_y) = target_ids.shape
    binary_labels = np.zeros(shape=(dim_x, dim_y, num_classes))
    for class_id in range(num_classes):
        binary_labels[:, :, class_id] = target_ids == class_id
    return jnp.array(binary_labels)


In [29]:
def prep_data(data, height, width, num_classes):
    """
    Prepares the data by resizing to the requested height/width and
    binarizing the target ID's.
    Args:
        data: Dictionary containing a single input image and annotation.
        height: Height dimension for resizing the images.
        width: Width dimension for resizing the images.
        num_classes: Total number of distinct classes.
    Returns:
        The resized input image and resized binarized target ID's.
    """
    input, target_ids = jnp.array(data["image"][0]), np.array(data["annotation"][0])
    target_ids_binary = binarize_target(target_ids, num_classes)
    # Downsample the image
    input = jax.image.resize(input, shape=(height, width, 3), method="bilinear")
    target_ids_binary = jax.image.resize(target_ids_binary, shape=(height, width, num_classes), method="nearest")
    return input, target_ids_binary
    

In [30]:
def prep_data_batch(data_generator, batch_size, height, width, num_classes):
    """
    Prepares a batch of data for training/validation.
    Args:
        data_generator: Generator object containing the 
            training/validation data.
        batch_size: Number of data to process in parallel. 
        height: Height dimension for resizing the images.
        width: Width dimension for resizing the images.
        num_classes: Total number of distinct classes.
    Returns:
        A Jax array of input images and target images.
    """
    inputs = []
    targets = []
    for _ in range(batch_size):
        data = next(data_generator)
        input, target_ids_binary = prep_data(data, height, width, num_classes)
        inputs.append(input)
        targets.append(target_ids_binary)
    inputs = jnp.array(inputs)
    targets = jnp.array(targets)
    return inputs, targets

In [31]:
@jax.jit
def dice_loss(variables, inputs, targets, epsilon):
    """
    Calculates the dice loss for a batch of images.
    Args:
        variables: The segmentation models parameters.
        inputs: A batch of raw input images.
        targets: A batch of arrays with the same shape as inputs. Stores the binary
                 classification label for each element in inputs
                (0 for the negative class and 1 for the positive class).
        epsilon: A small number to prevent loss function from dividing by zero.
    Returns:
        The dice loss from a batch of images.
    """
    preds = unet.apply(variables, inputs)
    numerator = 2 * (preds * targets).sum(axis=(-2, -3))
    denominator = preds.sum(axis=(-2, -3)) + targets.sum(axis=(-2, -3))
    loss = 1 - jnp.mean((numerator + epsilon) / (denominator + epsilon))
    return loss

In [44]:
# Number of training epochs
epochs = 10
# Validation frequency
val_epochs = 10
# Learning rate
lr = 0.01
batch_size = 1
# A small number to prevent loss function from dividing by zero
epsilon = 0.0001


variables = unet.init(rng_key, dummy_x)
optimizer = optax.adam(learning_rate=lr)
opt_state = optimizer.init(variables)

train_data_generator = train_dataset.iter(batch_size=1)
val_data_generator = val_dataset.iter(batch_size=1)

losses = []

for epoch in range(epochs):
    print(f"Epoch: {epoch}")
    # Prepare a batch of data
    inputs, targets = prep_data_batch(train_data_generator, batch_size, height, width, num_classes)
    # Perform backpropagation 
    loss, grads = value_and_grad(dice_loss, argnums=(0))(variables, inputs, targets, epsilon)
    updates, opt_state = optimizer.update(grads, opt_state, variables)
    variables = optax.apply_updates(variables, updates) 
    # Evaluate the models performance
    if epoch % val_epochs == 0:
        print(f"Validation ")
        # Prepare a batch of data
        val_inputs, val_targets = prep_data_batch(val_data_generator, batch_size, height, width, num_classes)


    losses.append(loss)
    print(f"\tLoss: {loss :.2f}")


    

Epoch: 0
Validation 
	Loss: 0.99
Epoch: 1
	Loss: 0.68
Epoch: 2
	Loss: 0.63
Epoch: 3
	Loss: 0.65
Epoch: 4
	Loss: 0.62
Epoch: 5
	Loss: 0.61
Epoch: 6
	Loss: 0.60
Epoch: 7
	Loss: 0.61
Epoch: 8
	Loss: 0.59
Epoch: 9
	Loss: 0.59
